# Modeling

This notebook is based on Kaggle <a href="https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard">kernel</a>.

Interesting moments:
- Convert numerical features to category
- Represent year and month as category
- New feature TotalSF that is sum of 3 feature
- Used Box-Cox transforming. <a href="http://onlinestatbook.com/2/transformations/box-cox.html">Link to paper</a>

In [2]:
#import some necessary librairies
import sys
sys.path.append("../../dstoolkit/")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns

color = sns.color_palette()
sns.set_style('darkgrid')

import warnings

def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points